# Demo: AUC confidence intervals using Monte Carlo method
This notebook demonstrates how to use the `crv.validation.monte_carlo.MonteCarloPD` class to calculate some confidence intervals for the `auc` test.

## 0. Imports

In [1]:
# Imports.
import numpy as np
import pandas as pd

from crv.validation.monte_carlo import MonteCarloPD
from crv.validation.differentiation import auc
from crv.validation.summary import adf_summary
from multiprocessing import Pool, cpu_count

from IPython.display import display

Invoking __init__.py for crv.utils


---
## 1. Sample data
The following `pandas.DataFrame` is created from an ECB template.

In [2]:
# Helper functions and data generation.
def get_ecb_adf_table():
    d_adf = {'rat_rank': list(range(17)),
             'Rating': [str(i+1) for i in range(17)],
             'N': [105_000, 110_000, 100_000, 80_000, 60_000,
                   120_000, 135_000, 40_000, 30_000, 75_000,
                   55_000, 40_000, 30_000, 20_000, 10_000,
                   5_000, 2_000],
             'D': [10, 20, 20, 30, 50, 100, 150, 90, 50, 350, 550,
                   750, 900, 1000, 900, 700, 1500],
             'PD': [0.0001, 0.00015, 0.0002, 0.0004, 0.0008, 0.0009,
                    0.001, 0.0015, 0.002, 0.005, 0.01, 0.02, 0.03,
                    0.05, 0.1, 0.15, 0.5],
             'ORGEXP': [400_000_000, 440_000_000, 390_000_000, 315_000_000,
                        235_000_000, 470_000_000, 530_000_000, 150_000_000,
                        120_000_000, 300_000_000, 210_000_000, 160_000_000,
                        120_000_000, 80_000_000, 40_000_000, 20_000_000,
                        8_000_000]
             }
    df = pd.DataFrame.from_dict(d_adf)
    df = df.sort_values('rat_rank')
    return df

In [3]:
df = get_ecb_adf_table()
display(df)

rat_rank Rating       N     D       PD     ORGEXP
0          0      1  105000    10  0.00010  400000000
1          1      2  110000    20  0.00015  440000000
2          2      3  100000    20  0.00020  390000000
3          3      4   80000    30  0.00040  315000000
4          4      5   60000    50  0.00080  235000000
5          5      6  120000   100  0.00090  470000000
6          6      7  135000   150  0.00100  530000000
7          7      8   40000    90  0.00150  150000000
8          8      9   30000    50  0.00200  120000000
9          9     10   75000   350  0.00500  300000000
10        10     11   55000   550  0.01000  210000000
11        11     12   40000   750  0.02000  160000000
12        12     13   30000   900  0.03000  120000000
13        13     14   20000  1000  0.05000   80000000
14        14     15   10000   900  0.10000   40000000
15        15     16    5000   700  0.15000   20000000
16        16     17    2000  1500  0.50000    8000000

---
## 2. `MonteCarloPD` object and confidence intervals
The `crv.valiation.monte_carlo.MonteCarloPD` class contains the functionality to generate the simulation data based on user inputs. The function is designed to accept test functions that calculate and return a single metric, but the only library function that currently works is the `crv.validattion.differentiation.auc` test (which is set as the default test).

In [5]:
# Create the MonteCarloPD object.
mc = MonteCarloPD(data=df, default_count_col='D', customer_count_col='N',
                  pd_col='PD', test_func=auc)
mc

In [6]:
# Run a number of trials. 2000 rounds takes roungly 50 seconds on a 4-core laptop.
mc.run(num_rounds=2_000)

|#########################| Completed 2000 of 2000 simulation rounds.

Execution time: 1 minutes 19.5 seconds


In [7]:
# Get the test results for the orgininal data.
mc.get_test_results()

0.9297442334322231

In [8]:
# Get the confidence intervals on the test results, specifying alpha.
mc.get_ci(alpha=0.95)

array([0.92335396, 0.92929384])